In [ ]:
# !pip install transformers
# !pip install datasets
# !pip install keras-tuner
# !pip install evaluate

In [ ]:
# !pip install -U tensorboardx
# !pip install boto3



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [80]:
import os
import pandas as pd
import numpy as np
import pickle
import string
import re
import boto3

from sklearn.feature_extraction.text import TfidfVectorizer
import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE


import nltk
from nltk.corpus import stopwords, wordnet 
from nltk.tokenize import punkt  
from nltk.stem import WordNetLemmatizer
# from nltk.classify.api import MultiClassifierI

from absl import logging
import tensorflow as tf
import tensorflow_hub as hub


import torch
import torch.nn as nn
import torch.optim as optim


import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding, EarlyStoppingCallback, pipeline
import warnings
import time

from datasets import load_dataset, load_metric

import evaluate

# import ray
# from ray import tune
# from ray.tune.schedulers import PopulationBasedTraining

# import torch_geometric.transforms as T
# from torch_geometric.datasets import OGB_MAG
# from torch_geometric.nn import HeteroConv, GCNConv, SAGEConv, GATConv, Linear


In [ ]:
emotion = load_dataset("emotion")
emotion

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Dataset emotion downloaded and prepared to /root/.cache/huggingface/datasets/emotion/default/0.0.0/348f63ca8e27b3713b6c04d723efe6d824a56fb3d1449794716c0f0296072705. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [ ]:
#set train/test/val data
train_emote_small = emotion["train"].shuffle(seed=5).select([i for i in list(range(8000))])
test_emote_small = emotion["test"].shuffle(seed=5).select([i for i in list(range(1000))])
val_emote_small = emotion["validation"].shuffle(seed=5).select([i for i in list(range(1000))])


In [ ]:
train_emote = pd.DataFrame(train_emote_small, columns=['text', 'label'])
test_emote = pd.DataFrame(test_emote_small, columns=['text', 'label'])
val_emote = pd.DataFrame(val_emote_small, columns=['text', 'label'])

In [ ]:
train_emote

,text,label
0,i feel like i am being one person whom his lif...,0
1,i remember feeling equally dazed and road roll...,5
2,ive been frustrated that i dont walk around fl...,4
3,i learned to feel the clay and its limits the ...,1
4,i breaking skin feels like and it s not pleasant,1
...,...,...
7995,i feel that supporting or at least not condemn...,2
7996,i feel like any student response can tip the d...,2
7997,i can recall feelings of the time we were in c...,5
7998,i cant stop feeling delighted with myself,1


In [107]:
# from nltk.chunk import accuracy
metric= load_metric('accuracy')

def encode_data(data):
  tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
  return tokenizer(data["text"], truncation = True)

def model_init():
  return AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=6)


def model_metrics(eval_pred):
  metric= load_metric('accuracy')  
  logits, labels = eval_pred
  predictions = logits.argmax(axis=-1)
  return metric.compute(predictions=predictions, references=labels)

token_data = emotion.map(encode_data, batched=True)
token_train_sample = token_data["train"].shuffle(seed=5).select([i for i in list(range(8000))])
token_validation_sample = token_data["validation"].shuffle(seed=5).select([i for i in list(range(1000))])

In [ ]:
token_data

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 2000
    })
})

In [ ]:

# data = load_dataset("", split="validation").shuffle().select(1000)
# task_evaluator = evaluator("token-classification")

# results = []
# for model in models:
#     results.append(
#         task_evaluator.compute(
#             model_or_pipeline=model, data=data, metric="seqeval"
#             )
#         )

# df = pd.DataFrame(results, index=models)
# df[["overall_f1", "overall_accuracy", "total_time_in_seconds", "samples_per_second", "latency_in_seconds"]]

In [108]:
from ray.air import session
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

dir = "models/"
train_param = TrainingArguments(output_dir=".",  
                                evaluation_strategy="epoch", 
                                logging_strategy="epoch",
                                disable_tqdm=True,
                                do_eval=True,
                                save_strategy='epoch',
                                num_train_epochs=10,
                                logging_steps=1,
                                logging_first_step=False,
                                overwrite_output_dir=True,
                                weight_decay = 0.01,
                                per_device_train_batch_size=64,
                                per_device_eval_batch_size=64,
                                load_best_model_at_end=True,
                                metric_for_best_model='accuracy'
                                )

def hp_space1(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [8, 32, 64]),
        "weight_decay": trial.suggest_float("weight_decay", 1e-9, 1e-1, log=True)}

trainer = Trainer(
    model_init=model_init,
    args=train_param,
    data_collator=DataCollatorWithPadding(tokenizer),
    train_dataset=token_data["train"].shuffle(seed=5).select([i for i in list(range(8000))]), 
    eval_dataset=token_data["validation"].shuffle(seed=5).select([i for i in list(range(1000))]),
    compute_metrics = model_metrics,
    # stop if model does not imporove for 3 epochs
    callbacks=[EarlyStoppingCallback(3, 0.0)]
    
)


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.25.1",
  "vocab_size": 30522
}

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/vocab.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapsh

In [109]:
t1 = trainer.hyperparameter_search(direction="maximize", n_trials=10)

No `resources_per_trial` arg was passed into `hyperparameter_search`. Setting it to a default value of 1 CPU and 1 GPU for each trial.


== Status ==
Current time: 2022-12-06 22:00:38 (running for 00:00:00.19)
Memory usage on this node: 7.4/83.5 GiB 
Using FIFO scheduling algorithm.
Resources requested: 1.0/12 CPUs, 1.0/1 GPUs, 0.0/49.67 GiB heap, 0.0/24.83 GiB objects
Result logdir: /root/ray_results/_objective_2022-12-06_22-00-38
Number of trials: 10/10 (9 PENDING, 1 RUNNING)
+------------------------+----------+------------------+-----------------+--------------------+------------------------+----------+
| Trial name             | status   | loc              |   learning_rate |   num_train_epochs |   per_device_train_bat |     seed |
|                        |          |                  |                 |                    |                ch_size |          |
|------------------------+----------+------------------+-----------------+--------------------+------------------------+----------|
| _objective_6b3c4_00000 | RUNNING  | 172.28.0.12:7195 |     5.61152e-06 |                  5 |                     64 |  8.15

(_objective pid=7195) Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
(_objective pid=7195) - This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
(_objective pid=7195) - This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
(_objective pid=7195) Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-unc

== Status ==
Current time: 2022-12-06 22:00:48 (running for 00:00:10.13)
Memory usage on this node: 8.7/83.5 GiB 
Using FIFO scheduling algorithm.
Resources requested: 1.0/12 CPUs, 1.0/1 GPUs, 0.0/49.67 GiB heap, 0.0/24.83 GiB objects
Result logdir: /root/ray_results/_objective_2022-12-06_22-00-38
Number of trials: 10/10 (9 PENDING, 1 RUNNING)
+------------------------+----------+------------------+-----------------+--------------------+------------------------+----------+
| Trial name             | status   | loc              |   learning_rate |   num_train_epochs |   per_device_train_bat |     seed |
|                        |          |                  |                 |                    |                ch_size |          |
|------------------------+----------+------------------+-----------------+--------------------+------------------------+----------|
| _objective_6b3c4_00000 | RUNNING  | 172.28.0.12:7195 |     5.61152e-06 |                  5 |                     64 |  8.15

(_objective pid=7195) <ipython-input-107-246380b83a36>:13: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate


...

== Status ==
Current time: 2022-12-06 22:14:02 (running for 00:13:23.96)
Memory usage on this node: 8.6/83.5 GiB 
Using FIFO scheduling algorithm.
Resources requested: 1.0/12 CPUs, 1.0/1 GPUs, 0.0/49.67 GiB heap, 0.0/24.83 GiB objects
Result logdir: /root/ray_results/_objective_2022-12-06_22-00-38
Number of trials: 10/10 (1 RUNNING, 9 TERMINATED)
+------------------------+------------+------------------+-----------------+--------------------+------------------------+----------+-------------+
| Trial name             | status     | loc              |   learning_rate |   num_train_epochs |   per_device_train_bat |     seed |   objective |
|                        |            |                  |                 |                    |                ch_size |          |             |
|------------------------+------------+------------------+-----------------+--------------------+------------------------+----------+-------------|
| _objective_6b3c4_00009 | RUNNING    | 172.28.0.12:8040 | 

(raylet) [2022-12-06 22:14:12,283 E 970 988] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-12-06_18-22-53_980285_113 is over 95% full, available space: 2763567104; capacity: 83955703808. Object creation will fail if spilling is required.


(_objective pid=8040) {'loss': 0.8792, 'learning_rate': 0.0, 'epoch': 2.0}
== Status ==
Current time: 2022-12-06 22:14:12 (running for 00:13:33.97)
Memory usage on this node: 8.6/83.5 GiB 
Using FIFO scheduling algorithm.
Resources requested: 1.0/12 CPUs, 1.0/1 GPUs, 0.0/49.67 GiB heap, 0.0/24.83 GiB objects
Result logdir: /root/ray_results/_objective_2022-12-06_22-00-38
Number of trials: 10/10 (1 RUNNING, 9 TERMINATED)
+------------------------+------------+------------------+-----------------+--------------------+------------------------+----------+-------------+
| Trial name             | status     | loc              |   learning_rate |   num_train_epochs |   per_device_train_bat |     seed |   objective |
|                        |            |                  |                 |                    |                ch_size |          |             |
|------------------------+------------+------------------+-----------------+--------------------+------------------------+----------

2022-12-06 22:14:14,851	INFO tune.py:777 -- Total run time: 816.15 seconds (816.02 seconds for the tuning loop).


== Status ==
Current time: 2022-12-06 22:14:14 (running for 00:13:36.03)
Memory usage on this node: 8.6/83.5 GiB 
Using FIFO scheduling algorithm.
Resources requested: 0/12 CPUs, 0/1 GPUs, 0.0/49.67 GiB heap, 0.0/24.83 GiB objects
Result logdir: /root/ray_results/_objective_2022-12-06_22-00-38
Number of trials: 10/10 (10 TERMINATED)
+------------------------+------------+------------------+-----------------+--------------------+------------------------+----------+-------------+
| Trial name             | status     | loc              |   learning_rate |   num_train_epochs |   per_device_train_bat |     seed |   objective |
|                        |            |                  |                 |                    |                ch_size |          |             |
|------------------------+------------+------------------+-----------------+--------------------+------------------------+----------+-------------|
| _objective_6b3c4_00000 | TERMINATED | 172.28.0.12:7195 |     5.61152e-0

In [ ]:
t1

BestRun(run_id='02b6b_00005', objective=0.938, hyperparameters={'learning_rate': 1.1207606211860571e-05, 'num_train_epochs': 4, 'seed': 1.8994345766152145, 'per_device_train_batch_size': 16})

In [ ]:
param2 = t1.hyperparameters
param2

{'learning_rate': 1.5620693675639888e-05,
 'num_train_epochs': 2,
 'seed': 7.0837862931119036,
 'per_device_train_batch_size': 16}

In [ ]:
trainer.train()

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.25.1",
  "vocab_size": 30522
}



{'loss': 0.8446, 'learning_rate': 4.5e-05, 'epoch': 1.0}


Saving model checkpoint to ./checkpoint-125
Configuration saved in ./checkpoint-125/config.json


{'eval_loss': 0.31042101979255676, 'eval_accuracy': 0.902, 'eval_runtime': 0.6974, 'eval_samples_per_second': 1433.974, 'eval_steps_per_second': 22.944, 'epoch': 1.0}


Model weights saved in ./checkpoint-125/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 64


...

Saving model checkpoint to ./checkpoint-750
Configuration saved in ./checkpoint-750/config.json


{'eval_loss': 0.24711443483829498, 'eval_accuracy': 0.936, 'eval_runtime': 0.6949, 'eval_samples_per_second': 1439.115, 'eval_steps_per_second': 23.026, 'epoch': 6.0}


Model weights saved in ./checkpoint-750/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./checkpoint-375 (score: 0.938).


{'train_runtime': 82.8305, 'train_samples_per_second': 965.828, 'train_steps_per_second': 15.091, 'train_loss': 0.23298442268371583, 'epoch': 6.0}


TrainOutput(global_step=750, training_loss=0.23298442268371583, metrics={'train_runtime': 82.8305, 'train_samples_per_second': 965.828, 'train_steps_per_second': 15.091, 'train_loss': 0.23298442268371583, 'epoch': 6.0})

In [ ]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 64


{'eval_loss': 0.17113631963729858, 'eval_accuracy': 0.938, 'eval_runtime': 0.76, 'eval_samples_per_second': 1315.739, 'eval_steps_per_second': 21.052, 'epoch': 6.0}


{'eval_loss': 0.17113631963729858,
 'eval_accuracy': 0.938,
 'eval_runtime': 0.76,
 'eval_samples_per_second': 1315.739,
 'eval_steps_per_second': 21.052,
 'epoch': 6.0}

In [ ]:
# !zip -r model.zip model/

In [ ]:
# from google.colab import files
# files.download("/content/model.zip")

In [89]:
aa = pd.read_csv("arthur_andersen_filtered.csv")
# pre = pd.read_csv("pre_event.csv")
# post = pd.read_csv("post_event.csv")

In [92]:
def preprocess_txt(str):
  
    # attempts to remove email addresses and extraneous spaces in text body
  lines = str.split('\n')
  for line in lines:
    if ':' not in line:
      if not re.findall(r"(^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$)", line) and \
      not re.findall(r"(\.com|\.org)", line):
        str = line.strip()
        

    return str
  
  
# for body in post["Content"]:
#     preprocess_txt(body)

for body in aa["Content"]:
    preprocess_txt(body)


  

In [105]:
aa.shape

(3319, 6)

In [ ]:
# vectorizer = feature_extraction.text.CountVectorizer(max_features=10000, ngram_range=(1,2))

In [ ]:
# BERT tuple input structure = [cls] + sentence + [sep] + sentence [sep]...
# artifacts from splitting content body and random spaces confused the model here, when the variable was passed to tokenizer
# X_pre_tokened = [" [CLS] " + s + " [SEP] " for s in X_pre]

In [82]:
# load Universal Sentence Encoder module
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(module_url)

def embedding(_):
  return model(_)

In [98]:
embedding_tuples = []
labels = aa['Poi']

for c , label in zip(aa['Content'], labels):
  tmp = embedding([c])[0].numpy(),label
  embedding_tuples.append(tmp)


In [103]:
aa_tup = tuple(embedding_tuples)

In [104]:
len(aa_tup)

3319

In [106]:
train.evaluate()

NameError: ignored

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
X_pre_tokened = [tokenizer.tokenize(pre) in X_pre]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.25.1",
  "vocab_size": 30522
}

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/vocab.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapsh

TypeError: ignored